# 2D Convolution With Stride & Padding

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
import os
import cv2

In [3]:
import tensorflow as tf

### Reading data as image and resizing and test, train & validation separated

In [110]:
def read_data(path):
    
    train_path = path+"/train"
    test_path = path+"/test"
    validation_path = path+"/val"
    
    tr_data, test_data, val_data = [], [], []
    tr_out, test_out, val_out = [], [], []

    for i in os.listdir(train_path):

        try:
            for j in os.listdir(train_path+"/"+i):
                image = cv2.imread(train_path+"/"+i+"/"+j, cv2.IMREAD_GRAYSCALE)
                tr_data.append(cv2.resize(image, (224, 224)))
                tr_out.append(i)

            for j in os.listdir(test_path+"/"+i):
                image = cv2.imread(test_path+"/"+i+"/"+j, cv2.IMREAD_GRAYSCALE)
                test_data.append(cv2.resize(image, (224, 224)))
                test_out.append(i)

            for j in os.listdir(validation_path+"/"+i):
                image = cv2.imread(validation_path+"/"+i+"/"+j, cv2.IMREAD_GRAYSCALE)
                val_data.append(cv2.resize(image, (224, 224)))
                val_out.append(i)
        except:
            pass
                
                
    tr_data, test_data, val_data = np.array(tr_data), np.array(test_data), np.array(val_data)
    tr_out, test_out, val_out = np.array(tr_out), np.array(test_out), np.array(val_out)

    return tr_data, test_data, val_data, tr_out, test_out, val_out



In [111]:
path = "/Users/rajeshr/Desktop/Group_22"
tr_data, test_data, val_data, tr_out, test_out, val_out = read_data(path)

In [4]:
path = '/Users/rajeshr/Desktop/Group_22/selected/'

images = []
label = []
for i in os.listdir(path):
    image = cv2.imread(path+i, cv2.IMREAD_GRAYSCALE)
    try:
        images.append(cv2.resize(image, (224, 224)))
        label.append(i)
    except:
        pass
    
images = np.array(images)
label = np.array(label)

## Kaiming Initializatied Filter

In [5]:
def kaiming(n, size):
    mean, sd = 0, np.sqrt(2/n)
    weights = np.random.normal(mean, sd, size=size)
    return weights

## Convolutional Filter

In [7]:
def zero_padding(inpt, padding):
    
    m, n = inpt.shape
    
    temp = []
    for i in range(padding):
        for j in range(m+2*padding):
            temp.append(0)
            
    for i in inpt:
        
        for j in range(padding):
            temp.append(0)
        
        for p in i:
            temp.append(p)
            
        for k in range(padding):
            temp.append(0)
            
    for i in range(padding):
        for j in range(m+2*padding):
            temp.append(0)
    
    
    temp = np.array(temp, dtype=np.uint8)
    temp = temp.flatten()
    temp = temp.reshape(m+2*padding, n+2*padding)
    
    return temp

## Without Depth

In [8]:
def convolution(inpt, filtr , stride, padding):
    
    (Am, An), (fm, fn) = inpt.shape, filtr.shape
    inpt = zero_padding(inpt, padding)
    m, n = inpt.shape
    
    feature_map = []
    
    flip_leftright = np.fliplr(filtr)
    flip_updown = np.flipud(flip_leftright)
    Filtr = flip_updown 
    
    Outm, Outn = int(((Am-fm+2*padding)/stride)+1), int(((An-fn+2*padding)/stride)+1)
    
    feature_map = []
    for i in range(0, m, stride):
        for j in range(0, n, stride):
            if inpt[i:i+fn, j:j+fm].shape == Filtr.shape:
                feature_map.append((Filtr*inpt[i:i+fn, j:j+fm]).sum())

    feature_map = np.array(feature_map).reshape(Outm, Outn)
    return feature_map

In [20]:
n = 9
filtr = kaiming(n, (3, 3))
np.savetxt('/Users/rajeshr/Desktop/filter.txt', filtr, delimiter=',')
cv2.imwrite('/Users/rajeshr/Desktop/filter.jpg', filtr)

for image in images:
    cv2.imwrite('/Users/rajeshr/Desktop/input'+str(np.random.randint(1000))+'.jpg', image)

for image in images:
    feature_map = convolution(image, filtr, stride = 1, padding = 0)
    np.savetxt('/Users/rajeshr/Desktop/featuremap.txt', feature_map, delimiter=',')
    cv2.imwrite('/Users/rajeshr/Desktop/feature_map'+str(np.random.randint(1000))+'.jpg', feature_map)

## With Depth

In [115]:
def convwithDepth(inpt=image, filtr=filtr, stride = 1, padding = 0):
    print(inpt.shape, filtr.shape)
    (D, Am, An), (fm, fn) = inpt.shape, filtr.shape
    
    padded_inpt = []
    for i in inpt:
        padded_inpt.append(zero_padding(i, padding))
        
    inpt = np.array(padded_inpt)
    print(inpt.shape)
    d, m, n = inpt.shape
    
    flip_leftright = np.fliplr(filtr)
    flip_updown = np.flipud(flip_leftright)
    Filtr = flip_updown 
    
    Outm, Outn = int(((Am-fm+2*padding)/stride)+1), int(((An-fn+2*padding)/stride)+1)
    #print(Outm, Outn)
    
    feature_map = []
    for i in range(0, m, stride):
        for j in range(0, n, stride):
            if inpt[:, i:i+fn, j:j+fm][0].shape == Filtr.shape:
                feature_map.append((Filtr*inpt[:, i:i+fn, j:j+fm]).sum())

    feature_map = np.array(feature_map).reshape(Outm, Outn)
    return feature_map

In [119]:
feature_map = convwithDepth(inpt=tr_data, filtr=filtr, stride = 1, padding = 0)
print(feature_map.shape)
cv2.imwrite('/Users/rajeshr/Desktop/feature_map'+str(np.random.randint(1000))+'.jpg', feature_map)

(150, 224, 224) (3, 3)
(150, 224, 224)
(222, 222)


True